In [3]:
from faker import Faker
import random
import sqlite3
import pandas as pd

# Initialize the Faker library
fake = Faker()

# Create a database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Create Customers table
cursor.execute('''CREATE TABLE IF NOT EXISTS Customers (
                    customer_id INTEGER PRIMARY KEY,
                    name TEXT,
                    shipping_addr TEXT,
                    contact_number TEXT,
                    current_flag INTEGER DEFAULT 1
                )''')

# Create Orders table
cursor.execute('''CREATE TABLE IF NOT EXISTS Orders (
                    order_id INTEGER PRIMARY KEY,
                    customer_id INTEGER,
                    order_date TEXT,
                    total_amount REAL NOT NULL,
                    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
                )''')

# Create Order Items table
cursor.execute('''CREATE TABLE IF NOT EXISTS OrderItems (
                    item_id INTEGER PRIMARY KEY,
                    order_id INTEGER,
                    product_id INTEGER,
                    quantity INTEGER,
                    price REAL,
                    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
                    FOREIGN KEY (product_id) REFERENCES Products(product_id)
                )''')

# Create Products table
cursor.execute('''CREATE TABLE IF NOT EXISTS Products (
                    product_id INTEGER PRIMARY KEY,
                    name TEXT,
                    description TEXT
                )''')

# Create Variants table
cursor.execute('''CREATE TABLE IF NOT EXISTS Variants (
                    variant_id INTEGER PRIMARY KEY,
                    name TEXT,
                    parent_product_id INTEGER,
                    FOREIGN KEY (parent_product_id) REFERENCES Products(product_id)
                )''')

# Generate and insert customers
for _ in range(10):
    name = fake.name()
    shipping_addr = fake.address().replace("\n", ", ")
    contact_number = fake.phone_number()
    cursor.execute('''INSERT INTO Customers (name, shipping_addr, contact_number)
                      VALUES (?, ?, ?)''', (name, shipping_addr, contact_number))
conn.commit()

# Generate and insert products
product_names = ['iPhone', 'MacBook', 'Apple Watch', 'AirPods', 'Apple TV', 'iPod', 'Beats', 'iPad', 'HomePod', 'iMac']
for name in product_names:
    description = fake.sentence(nb_words=6)
    cursor.execute('''INSERT INTO Products (name, description)
                      VALUES (?, ?)''', (name, description))
conn.commit()

# Generate and insert product variants
variant_names = ['iPhone 13', 'iPhone 14', 'MacBook Air', 'MacBook Pro']
parent_ids = [1, 1, 2, 2]  # Assuming iPhone has id 1 and MacBook has id 2
for i in range(len(variant_names)):
    cursor.execute('''INSERT INTO Variants (name, parent_product_id)
                      VALUES (?, ?)''', (variant_names[i], parent_ids[i]))
conn.commit()

# Generate and insert orders and order items
for _ in range(100):
    customer_id = random.randint(1, 10)
    order_date = fake.date_between(start_date='-2y', end_date='today')
    total_amount = round(random.uniform(10, 1000), 2)
    cursor.execute('''INSERT INTO Orders (customer_id, order_date, total_amount)
                      VALUES (?, ?, ?)''', (customer_id, order_date, total_amount))
    order_id = cursor.lastrowid

    # Generate order items
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        product_id = random.randint(1, 10)
        quantity = random.randint(1, 10)
        price = round(random.uniform(10, 100), 2)
        cursor.execute('''INSERT INTO OrderItems (order_id, product_id, quantity, price)
                          VALUES (?, ?, ?, ?)''', (order_id, product_id, quantity, price))
conn.commit()

# Close the initial connection
conn.close()

# Reopen the connection to retrieve data
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Function to fetch data and convert to DataFrame
def fetch_to_dataframe(query, conn):
    cursor.execute(query)
    data = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    return pd.DataFrame(data, columns=columns)

# Fetch and display the data
orders_df = fetch_to_dataframe("SELECT * FROM Orders", conn)
customers_df = fetch_to_dataframe("SELECT * FROM Customers", conn)
products_df = fetch_to_dataframe("SELECT * FROM Products", conn)
variants_df = fetch_to_dataframe("SELECT * FROM Variants", conn)
order_items_df = fetch_to_dataframe("SELECT * FROM OrderItems", conn)


# Close the connection
conn.close()


In [2]:
orders_df.head()

,order_id,customer_id,order_date,total_amount
0,1,10,2023-08-19,954.34
1,2,8,2024-04-30,32.93
2,3,5,2024-06-03,239.92
3,4,8,2024-03-15,60.57
4,5,6,2024-01-13,522.10
